In [ ]:


import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
data_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
data_rec = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
print(data.head(10))
print(data_deaths.head(10))
print(data_rec.head(10))
print(data.shape)
print(data_deaths.shape)
print(data_rec.shape)


In [ ]:
 pip install folium

In [ ]:
import folium
from folium.plugins import MarkerCluster
import numpy as np

In [ ]:
data['Total Confirmed Cases']= data.iloc[:, -1]

In [ ]:
print(data[data['Country/Region'] == 'United Kingdom'])
locations = data[['Lat', 'Long']]
locationlist = locations.values.tolist()

len(locationlist)




***Visualization: ***

After running the below code we will get a map.If we click on the icons on the map, it will get zoomed into the map and countries will appear. We can select which ever country we want. Upon hovering on each country we get the total number of confirmed cases and deaths in that particular country. If we click that country we get graphs showing trend in deaths and number of confirmed corona cases from start till yesterday.

In [ ]:
from folium.features import DivIcon
map2 = folium.Map(location=[0, 0], tiles='CartoDB dark_matter', zoom_start=2)

marker_cluster = MarkerCluster().add_to(map2)

def add_graphs(point):  
  import altair as alt
  y = list(data.iloc[point,4:].values)
  y_deaths = list(data_deaths.iloc[point,4:].values)
  a = len(y)+1
  c = len(y_deaths) + 1
  y = pd.DataFrame(y)
  #yinsert(1, "Deaths", y_deaths, True)
  y_deaths = pd.DataFrame(y_deaths)
  y.columns = ['Number']
  y['Days'] = range(1,a)
  y_deaths.columns = ['Number']
  y_deaths['Days'] = range(1,c)
  y_deaths['Legend'] = 'Confirmed Deaths'
  y['Legend'] = 'Confirmed Cases'
  #print(y.shape)

  #py.append(y_deaths,ignore_index=True)
  df_mix = pd.concat([y, y_deaths])
  #print(df_mix .shape)
  #print(df_mix .head(300))
  b = alt.Chart(df_mix).mark_line().encode(x='Days',y='Number',color = 'Legend')
  #b = alt.Chart(y).mark_line().encode(x='Days',y='Cases')
  # = alt.Chart(y_deaths).mark_line(color = 'red').encode(x='Days',y='Cases')
  #e = b + d
  b = b.to_json('vega.json')
  return b

for point in range(0, len(locationlist)):
  if(str(data['Province/State'][point]) == 'nan'):
    folium.Marker(locationlist[point],icon=folium.Icon(color = 'black',icon_color='red', icon='asterisk', angle=0, prefix='fa'),popup=folium.Popup(max_width=500).add_child(folium.VegaLite(add_graphs(point), width=400, height=250)),tooltip=(data['Country/Region'][point] + ", " +'Cases: '+ str(data['Total Confirmed Cases'][point]))).add_to(marker_cluster)
  else:
    folium.Marker(locationlist[point],icon=folium.Icon(color = 'black',icon_color='red', icon='asterisk', angle=0, prefix='fa'),popup=folium.Popup(max_width=400).add_child(folium.VegaLite(add_graphs(point), width=400, height=250)), tooltip=(data['Country/Region'][point] + ", " + 'Province: ' + str(data['Province/State'][point])+", " +'Cases: '+ str(data['Total Confirmed Cases'][point]))).add_to(marker_cluster)
map2





In [ ]:
## RNN for Covid_19 Data
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from keras.callbacks import ModelCheckpoint, EarlyStopping
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset
dataset = data.iloc[223,4:].values
dataset = dataset.reshape(-1, 1)
dataset = dataset.astype('float32')
print("Data is ",dataset.shape)
# normalize the dataset
#scaler = MinMaxScaler(feature_range=(0, 1))
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
# split into train and test sets
train_size = int(len(dataset) * 0.95)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
print("Shapeeee. ",trainX.shape)
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=150, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

In [ ]:
plt.plot(scaler.inverse_transform(dataset),label='Original')
plt.plot(trainPredictPlot,label = 'Training')
plt.plot(testPredictPlot,label = 'Testing')
plt.ylabel('Confirmed Corona Cases in UK')
plt.xlabel('Days')
plt.legend(loc='upper left')
plt.show()

In [ ]:
## Prdeiction of Confirmed Corona Cases in UK for next 5 days 
print(len(testX))
print(testX[2])
test = testX[2]
pred = []
for i in range(5):
  x_input = test.reshape((1, 1, 1))
  fut = model.predict(x_input, verbose=0)
  test = fut
  nor= scaler.inverse_transform(fut)
  list(nor)
  print(type(nor))
  print(nor)
  pred.append(nor)
print(pred)
print(len(pred))

**Prediction of confirmed cases in UK for next 5 days**

In [ ]:
print(" Prediction of number of Confirmed Cases in UK for the next five days are the following:")
print(" Day 1=%d \n Day 2=%d \n Day 3=%d \n Day 4=%d \n Day 5=%d  " % (pred[4],pred[3],pred[2],pred[1],pred[0]))